In [41]:
import os
import sys
import pandas as pd
import cv2
import numpy as np
import torch
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

In [60]:
from abc import ABC

import torch.nn as nn
import torchvision.models as models


class ResNet(nn.Module, ABC):
    def __init__(self, _model_):
        super(ResNet, self).__init__()
        self.model = _model_
        self.cnn = nn.Sequential(*list(self.model.children())[:-1], nn.Flatten())
        self.fc = nn.Linear(512, 4)

    def forward(self, x):
        x = self.cnn(x)
        x = self.fc(x)
        return x


class Vgg16bn(nn.Module, ABC):
    def __init__(self, _model_):
        super(Vgg16bn, self).__init__()
        self.model = _model_
        self.cnn = nn.Sequential(*list(self.model.children())[:-1], nn.Flatten())
        self.fc = nn.Linear(25088, 4)

    def forward(self, x):
        x = self.cnn(x)
        x = self.fc(x)
        return x


class ResNeXt(nn.Module, ABC):
    def __init__(self, _model_):
        super(ResNeXt, self).__init__()
        self.model = _model_
        self.cnn = nn.Sequential(*list(self.model.children())[:-1], nn.Flatten())
        self.fc = nn.Linear(in_features=2048, out_features=4)

    def forward(self, x):
        x = self.cnn(x)
        x = self.fc(x)
        return x


def baseline(name, pretrained=True):
    """
    :param name: name of the model
    :param pretrained: use pretrained model or not
    :return: model
    """
    try:
        if name == 'resnet18':
            model = models.resnet18(pretrained=pretrained)
            return ResNet(model)
        elif name == 'resnet34':
            model = models.resnet34(pretrained=pretrained)
            return ResNet(model)
        elif name == 'vgg16':
            model = models.vgg16_bn(pretrained=pretrained)
            return Vgg16bn(model)
        elif name == "resnext50":
            model = models.resnext50_32x4d(pretrained=pretrained)
            return ResNeXt(model)
        elif name == "resnext101":
            model = models.resnext101_32x8d(pretrained=pretrained)
            return ResNeXt(model)

    except Exception as e:
        print(e)

In [126]:
def get_grad_cam(model, target_layer, image_path, target_category=None):
    """

    :param model: Model to perform gradcam on
    :param target_layer: target layer in the model
    :param image_path: rgb image path
    :param target_category: class of target
    :return: cam
    """
    out = os.path.join("../output", "saliency")
    os.makedirs(out, exist_ok=True)
    image_name = image_path.split(os.sep)[-1][:-4]
    rgb_image = cv2.imread(image_path, 1)
    rgb_image = cv2.resize(rgb_image, (360, 240))
    rgb_image = rgb_image[:, :, ::-1]
    rgb_image = np.float32(rgb_image) / 255
    input_tensor = preprocess_image(rgb_image,
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    
    # output
    output = model(input_tensor)
    zone = torch.argmax(output[0][:-2]).item()
    
    print(f"Target:{target_category}, Prediction: {zone}, Model Output:{output[0]}")
    

    cam = GradCAM(model=model, target_layer=target_layer, use_cuda=False)
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

    # Here grayscale_cam has only one image in the batch
    grayscale_cam = grayscale_cam[0, :]

    cam_image = show_cam_on_image(rgb_image, grayscale_cam, use_rgb=True)

    # cam_image is RGB encoded whereas "cv2.imwrite" requires BGR encoding.
    cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(out, image_name + "_saliency.jpg"), cam_image)



In [130]:
path = "../data/Non-Intersection/Outside Objects/Day/Batch 2/Images/2474_OO_D_9cfaaef522d219cde03ebe2f3bc1b83409e7f6e8efc44fd72a16c94d17f7766179c2584543e45238a8b2897d46eebcc2433cdb362e107af482044dd7a3454974_116.jpg"
weight = "../output/baseline/resnext50/weights/LR_0.001_OPT_AdamW_BATCH_32_SHAPE_(240, 360)_testing/LR_0.001_OPT_AdamW_BATCH_32_SHAPE_(240, 360)_testing_best_weight.pth"
net.load_state_dict(torch.load(weight, map_location="cpu"))
target = list(net.cnn.children())[-3][-1]
get_grad_cam(model=net, target_layer=target, image_path=path, target_category=0)

Target:0, Prediction: 0, Model Output:tensor([ 2.7983, -2.7095, -1.4295,  1.6025], grad_fn=<SelectBackward>)
